In [17]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
from holoviews import opts
from holoviews.plotting.util import process_cmap
import panel as pn
import numpy as np
from datetime import timedelta  
from scipy.optimize import curve_fit
from bokeh.models.formatters import DatetimeTickFormatter
import matplotlib.colors as mcolors
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

pn.extension()
pd.options.plotting.backend = 'holoviews'

hv.renderer('bokeh').theme = 'light_minimal' # Graph style
def_opts = {'width': 500, 'height': 400, 'padding': 0.1, 'shared_axes': False, 
            'yformatter': '%d','xformatter': DatetimeTickFormatter(days = '%b %d')}
opts.defaults(opts.Scatter(**def_opts), opts.Curve(line_width=2.5, **def_opts))

In [18]:
cmap = list(mcolors.TABLEAU_COLORS.values())  # List of Hexdec values corresponding to default colormap of Bokeh/mpl

def Plot_All (df, log = True, lin = True, kind='scatter', ylabel = 'Number of cases', yformatter = '%d'):
    '''
    Plots the data in the following way:
    One tab for each country and each tab can have two tabs for linear and log plot.
    The first tab is the plot of all Countries together
    
    ARGUMENTS:
    - df: Dataframe containing all data
    - log: set to False if you don't want the logarithmic scale tab
    - lin: set to False if you don't want the linear scale tab
    - kind: you can choose the plot kind (scatter by default)
    - ylabel: label of y axis
    '''    
    xformatter = DatetimeTickFormatter(days = '%b %d')   # Formatter for date time axis, so that date is e.g. 'Jan 25'
    
    opts_comm = {'title' : '', 'width' : 600, 'height':400, 'padding' : 0.1,   # Options in common between linear and log plots
                 'kind' : kind,  'xformatter': xformatter, 'xlabel': ''}
    opts_lin = {**opts_comm}    # Options for linear plots
    opts_log = {**opts_comm, **{'logy' : True, 'yformatter' : yformatter, 'ylim' : [9e-2, None]}}  # Options for log plots       
    
    # Create the first tab with all countries in the same plot
    if log == False: plt_all = df.plot(**opts_lin)
    elif lin == False: plt_all = df.plot(**opts_log)
    else: plt_all = pn.Tabs(('Linear', df.plot(**opts_lin)),
                            ('Logarithmic', df.plot(**opts_log)))
    tab_states = pn.Tabs(('All', plt_all))
    # Create other tabs, one for each country
    for i, Country in enumerate(df.columns):
        opts_lin['color'] = cmap[i]
        opts_log['color'] = cmap[i]
        if log == False: plt_country = df[Country].plot(**opts_lin)
        elif lin == False: plt_country = df[Country].plot(**opts_log)
        else: plt_country = pn.Tabs(('Linear', df[Country].plot(**opts_lin)),
                                    ('Logarithmic', df[Country].plot(**opts_log)))
        tab_states.append((Country, plt_country))
    return tab_states

# COVID-19 Data analysis
**Data source**: 
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series.
It's a repository from Johns Hopkins, which is updated around midnight each day.

**Source code that you can execute**:
https://mybinder.org/v2/gh/gioarma/covid-19_analysis/b5e55e36aa6ddaf4a797740d2fdfbb707ce901a1?filepath=Covid_19.ipynb

## Countries with most infected people

These are the 10 countries with most Coronavirus cases in the world.

In [19]:
data = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
df = pd.read_csv(data).T                                                        # Read the data transposing them
df.columns = [df.iloc[1,i] for i in range(len(df.columns))]                     # Change column names to Country
df.drop(['Province/State', 'Country/Region', 'Lat', 'Long'], inplace = True)    # Drop not needed columns
df.index = pd.to_datetime(df.index, format='%m/%d/%y')                          # Change index to DateTime for plotting
df = df.fillna(0.0).astype(int)                                                 # Convert data from float to int
df = df.groupby(level = 0, axis=1).sum()                                        # Sum all columns regarding the same state   

# Plot
Top_20 = df.iloc[-1, :].sort_values(ascending = False).head(20)
plt_Countries = Top_20.\
             plot(kind = 'bar', title = '', height = 500, legend = False, 
                  ylim = [0, df.max().max()*1.2], xformatter = '%d', hover = False).\
             opts(invert_axes=True, invert_yaxis = True)
overlay = plt_Countries
for i in range(20):
    overlay = overlay*hv.Text(x = Top_20.index[i], y = Top_20[i]+Top_20.max()*0.08, 
                              text = str("{:,}".format(Top_20[i])))
overlay

:Overlay
   .Bars.A_2020_hyphen_minus_03_hyphen_minus_30_00_colon_00_colon_00 :Bars   [index]   (2020-03-30 00:00:00)
   .Text.I                                                           :Text   [x,y]
   .Text.II                                                          :Text   [x,y]
   .Text.III                                                         :Text   [x,y]
   .Text.IV                                                          :Text   [x,y]
   .Text.V                                                           :Text   [x,y]
   .Text.VI                                                          :Text   [x,y]
   .Text.VII                                                         :Text   [x,y]
   .Text.VIII                                                        :Text   [x,y]
   .Text.IX                                                          :Text   [x,y]
   .Text.X                                                           :Text   [x,y]
   .Text.XI                                                          :Text   [x,y]
   .Text.XII                                                         :Text   [x,y]
   .Text.XIII                                                        :Text   [x,y]
   .Text.XIV                                                         :Text   [x,y]
   .Text.XV                                                          :Text   [x,y]
   .Text.XVI                                                         :Text   [x,y]
   .Text.XVII                                                        :Text   [x,y]
   .Text.XVIII                                                       :Text   [x,y]
   .Text.XIX                                                         :Text   [x,y]
   .Text.XX                                                          :Text   [x,y]

## Total Cases

Among all countries, we show the data for the following ones:

* China
* Italy
* France
* Germany
* Spain
* United States
* United Kingdom

The plot below shows the number of total cases in the selected countries.
You can view all of them in the same plot, or one by one by switching tabs.

You can also view each plot in linear or logarithmic scale.

In [20]:
Countries = ['Italy', 'China', 'France', 'Germany', 'Spain', 'US', 'United Kingdom']
TotCases = df.filter(Countries)
sorted_columns = list(TotCases.max().sort_values(ascending = False).index)
TotCases = TotCases[sorted_columns]
pn.Row(Plot_All(TotCases, kind = 'line'), pn.Spacer(width = 50),
        pn.pane.Markdown(""" 
        <br><br><br>
        ### Tip:
        You can activate the items in the side bar to move the plots, zoom and save the image."""))

Row
    [0] Tabs
        [0] Tabs
            [0] HoloViews(NdOverlay, name='Linear')
            [1] HoloViews(NdOverlay, name='Logarithmic')
        [1] Tabs
            [0] HoloViews(Curve, name='Linear')
            [1] HoloViews(Curve, name='Logarithmic')
        [2] Tabs
            [0] HoloViews(Curve, name='Linear')
            [1] HoloViews(Curve, name='Logarithmic')
        [3] Tabs
            [0] HoloViews(Curve, name='Linear')
            [1] HoloViews(Curve, name='Logarithmic')
        [4] Tabs
            [0] HoloViews(Curve, name='Linear')
            [1] HoloViews(Curve, name='Logarithmic')
        [5] Tabs
            [0] HoloViews(Curve, name='Linear')
            [1] HoloViews(Curve, name='Logarithmic')
        [6] Tabs
            [0] HoloViews(Curve, name='Linear')
            [1] HoloViews(Curve, name='Logarithmic')
        [7] Tabs
            [0] HoloViews(Curve, name='Linear')
            [1] HoloViews(Curve, name='Logarithmic')
    [1] Spacer(width=50)
    [2] Markdown(str)

## New cases

Here you can see how many *new* cases are recorded each day in the selected countries

In [21]:
NewCases = TotCases.diff().fillna(0.0).astype(int)                    # Calculate new cases as difference between successive rows, then convert to int

pn.Row(Plot_All(NewCases, log = False, kind = 'line', ylabel = 'Number of new cases'), 
       pn.Spacer(width = 50), 
       pn.pane.Markdown(""" 
        <br><br><br>
        ### Tip:
        In the "All" tab you can deactivate specific plots by clicking on the corresponding country in the side bar."""))

Row
    [0] Tabs
        [0] HoloViews(NdOverlay, name='All')
        [1] HoloViews(Curve, name='US')
        [2] HoloViews(Curve, name='Italy')
        [3] HoloViews(Curve, name='Spain')
        [4] HoloViews(Curve, name='China')
        [5] HoloViews(Curve, name='Germany')
        [6] HoloViews(Curve, name='France')
        [7] HoloViews(Curve, name='United Kingdom')
    [1] Spacer(width=50)
    [2] Markdown(str)

# Growth Factor

The growth factor is given by the number of new cases one day $N_d$ divided by the number of cases the previous day $N_{d-1}$:

$$G = \frac{N_d}{N_{d-1}}$$

When $G$ reaches the value of **1**, then we are probably at the inflection point, where the number of new cases every day starts to decrease

In [22]:
G = NewCases/NewCases.shift() #Divide by same df, but shifted back by 1
G.replace(np.inf, np.nan, inplace = True)
G.replace(0, np.nan, inplace = True)
panel_G = Plot_All(G.dropna(), log = True, ylabel = 'Growth factor', 
                   kind = 'scatter', lin = False, yformatter = '%.1f')
panel_G

Tabs
    [0] HoloViews(NdOverlay, name='All')
    [1] HoloViews(Scatter, name='US')
    [2] HoloViews(Scatter, name='Italy')
    [3] HoloViews(Scatter, name='Spain')
    [4] HoloViews(Scatter, name='China')
    [5] HoloViews(Scatter, name='Germany')
    [6] HoloViews(Scatter, name='France')
    [7] HoloViews(Scatter, name='United Kingdom')

# Italy

For this part we take data from Protezione Civile, which are more reliable.

Source: https://github.com/pcm-dpc/COVID-19/tree/master/dati-andamento-nazionale

In [23]:
dati_italia = 'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-andamento-nazionale/dpc-covid19-ita-andamento-nazionale.csv'
columns = ['data', 'totale_casi', 'deceduti', 'dimessi_guariti', 'totale_positivi']
Italia = pd.read_csv(dati_italia, usecols = columns, index_col='data')
Italia.index = pd.to_datetime(Italia.index)
Italia.columns = ['pos', 'rec', 'dead', 'tot']

# Calculate new columns
Italia['new'] = Italia.tot.diff()                                # New daily cases
Italia['new_pos'] = Italia.pos.diff()                            # New daily positives
Italia['new_dead'] = Italia.dead.diff()                          # New dead
Italia['new_rec'] = Italia.rec.diff()                            # New recovered

opts = {'width': 450, 'height': 400, 'xlabel': '', 'shared_axes': False,  'padding':0.1}
plt_TotItalia = Italia.tot.plot(label = 'Total Cases' , color = cmap[0], **opts)
plt_NewItalia = Italia.new.plot(label = 'New Cases',color = cmap[1], **opts)
plt_RecItalia = Italia.rec.plot(label = 'Total Recovered',color = cmap[2], **opts)
plt_NewRecItalia = Italia.new_rec.plot(label = 'New Daily Recovered',color = cmap[3], **opts)
plt_DeadItalia = Italia.dead.plot(label = 'Total Dead',color = cmap[4], **opts)
plt_NewDeadItalia = Italia.new_dead.plot(label = 'New Daily Dead',color = cmap[5], **opts)
plt_TotPosItalia = Italia.pos.plot(label = 'Total Currently Infected',color = cmap[6], **opts)
plt_NewPosItalia = Italia.new_pos.plot(label = 'New Daily Infected',color = cmap[7], **opts)

pltList_Italia = [plt_TotItalia, plt_NewItalia,
                  plt_TotPosItalia, plt_NewPosItalia,
                  plt_RecItalia, plt_NewRecItalia,
                  plt_DeadItalia, plt_NewDeadItalia]
hv.Layout(pltList_Italia).cols(2)

:Layout
   .Curve.Total_Cases              :Curve   [data]   (tot)
   .Curve.New_Cases                :Curve   [data]   (new)
   .Curve.Total_Currently_Infected :Curve   [data]   (pos)
   .Curve.New_Daily_Infected       :Curve   [data]   (new_pos)
   .Curve.Total_Recovered          :Curve   [data]   (rec)
   .Curve.New_Daily_Recovered      :Curve   [data]   (new_rec)
   .Curve.Total_Dead               :Curve   [data]   (dead)
   .Curve.New_Daily_Dead           :Curve   [data]   (new_dead)

## Predictions

This is based on the logistic model, which is the simplest model for epidemic studies. Therefore it should be considered for what it is: just as a very simple approximation of a very complicated process such as the spreading of a disease.

The idea is that the rate of change in the number of cases $C$ over time follows the law:

$$\frac{dC}{dt}=rC\left(1-\frac{C}{K}\right)$$

This differential equation has an analytical solution, which is a logistic curve:

$$C = \frac{K}{1+A\exp(-rt)}$$

Where $K$ is the number of expected final cases (also called final size), $r$ defines the spreading speed and $A$ is related to the time when the number of cases takes off.

Here I performa a non-linear fit to a logistic function with $K$, $r$ and $A$ as parameters (1st graph).
From the obtained curve it is possible to obtain the predicted number of _new_ cases by subtracting from the total cases on one day the ones on the previous day (2nd graph).

In [24]:
def sigmoid(t, K, A, r):
    return K/(1+A*np.exp(-r*(t)))

extra_days = 40    # How much we extend the fit beyond the current date for prediction
m = 10
N = Italia.index.size

C2m, Cm, C = Italia.tot[N-2*m], Italia.tot[N-m], Italia.tot[N-1]
K_guess = Cm*(C2m*Cm-2*C2m*C+Cm*C)/(Cm**2-C*C2m)
r_guess = 1/m*np.log(C*(Cm-C2m)/(C2m*(C-Cm)))
A_guess = (C-Cm)*(Cm-C2m)/(Cm**2-C*C2m)*(C/C2m*(Cm-C2m)/(C-Cm))**((N-1-m)/m)
init_guesses = [K_guess, A_guess, r_guess]   

# The index is in datetime format, with which I can't fit
# -> I consider days from beginning (0,1,2,3,...) as index
popt, pcov = curve_fit(sigmoid, [i for i in range(N)],
                       Italia.tot.values, 
                       p0 = init_guesses)

fit = pd.Series(index = pd.date_range(Italia.index[0], Italia.index[-1]+timedelta(days = extra_days), freq='D'), 
                data = sigmoid(np.arange(0, Italia.index.size+extra_days-1, 1), *popt))

opts = {'width': 600, 'height': 400, 'yformatter' : '%d', 'xlabel': ''}

plt_TotPred = Italia.tot.plot( kind = 'scatter', label = 'Data', **opts) * fit.plot(color = 'red', label = 'Prediction', hover = False, **opts) * \
              hv.HLine(popt[0]).opts(color = 'black', line_width=1, line_dash='dashed')* \
              hv.Text(text='  Predicted final size: '+str(int(popt[0])), y =popt[0]+5e3, x = fit.index[12], fontsize=11)* \
              hv.Arrow(Italia.index[-1]+timedelta(days=1), fit[Italia.index.size],
                       'Predicted for '+(Italia.index[-1]+timedelta(days=1)).strftime('%b %d')+': '+str(int(fit[Italia.index.size])), 
                       '^', arrowstyle = '-')
plt_NewPred = Italia.new.plot(kind = 'scatter', label = 'Data', c = cmap[1], **opts)*\
              fit.diff().plot(color = 'red', label = 'Prediction', hover = False)
(plt_TotPred.opts(title = 'Total cases') + plt_NewPred.opts(title = 'New cases')).cols(1)

:Layout
   .Overlay.I  :Overlay
      .Scatter.Data     :Scatter   [data]   (tot)
      .Curve.Prediction :Curve   [index]   (0)
      .HLine.I          :HLine   [x,y]
      .Text.I           :Text   [x,y]
      .Arrow.I          :Arrow   [x,y]
   .Overlay.II :Overlay
      .Scatter.Data     :Scatter   [data]   (new)
      .Curve.Prediction :Curve   [index]   (0)

### Is this converging?

The program fits daily the logistic function to the data, so each day the predicted curve changes. Therefore, it is relevant to see whether the curve in the last days has been varying a lot or if it is converging to a defined value.

Here I plot the predicted curve in different days, from today to 9 days ago.

In [25]:
#Italia.index = [i for i in range(len(Italia.index))]   # Convert the index to days since the beginning
J = 9                      # J will be the number of fitted curves: from index 0 to N-J, then to N-J+1, N-J+2, etc.
fits = pd.DataFrame(index = [i for i in range(N+1+extra_days)])
K_values = []
# In each cycle I perform the fit of the curve stopping at index n, with N-J<n<N-1
for n in range(N-J, N):
    # Find best initial guesses, based on (11), (12), (13) of [1]
    C2m, Cm, C = Italia.tot[n-2*m], Italia.tot[n-m], Italia.tot[n]
    K_guess = Cm*(C2m*Cm-2*C2m*C+Cm*C)/(Cm**2-C*C2m)
    r_guess = 1/m*np.log(C*(Cm-C2m)/(C2m*(C-Cm)))
    A_guess = (C-Cm)*(Cm-C2m)/(Cm**2-C*C2m)*(C/C2m*(Cm-C2m)/(C-Cm))**((n-2-m)/m)  # t_k = n-2. E.g. when n=N-1
    init_guesses = [K_guess, A_guess, r_guess]   
    
    # Fit
    popt, pcov = curve_fit(sigmoid, [i for i in range(n+1)],    # Have to put n+1 because in this cycle N-J<=n<=N, then when I use [] in a df, the last element is not considered
                           Italia.tot[0:n+1].values, 
                           p0 = init_guesses)
    K_values.append(popt[0])
    
    # Create fit curves that extend by 'extra_days' beyond the current date
    fit = pd.Series(index = [i for i in range(N+extra_days)], 
                             data = sigmoid(np.arange(0, N+extra_days), *popt))
    #fit.name = 'n = '+str(n)
    fit.name = Italia.index[n-N].strftime('%b %d')
    fits = fits.join(fit)        # I put all of them in a single df

plt_pred = Italia.tot.plot(kind = 'scatter')*fit.plot(color = 'red', label = 'Fit')
plt_pred.opts(padding = 0.1, width = 600, height = 400, yformatter = '%d', 
              xlabel = ' ')

fits = fits.iloc[:, ::-1]
fits.index = pd.date_range(start = Italia.index[0], 
                           end = Italia.index[-1]+timedelta(days = extra_days+2))
K = pd.DataFrame(data =K_values, columns=['K'], 
                 index = Italia.index[N-J:].strftime('%b %d'))

# Plot
cmap_fits = process_cmap('fire', ncolors=9)
fits.plot(color = cmap_fits, width = 700)

:NdOverlay   [Variable]
   :Curve   [index]   (value)

An easier way to visualize this is looking at the estimate for the final epidemic size $K$ for each of these predictions. If the values of $K$ oscillate a lot during the last 9 days, then probably we still haven't reached a stable situation and our prediction is not very reliable. 

In [26]:
K.plot.bar(label = 'Estimated final size of epidemic', ylabel = '', 
           xlabel = 'Days since Feb 24', yformatter = '%d', height = 400, hover = False)

:Bars   [data]   (K)

In [27]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>